# Homework 1

## Введение в OpenAI Gym

 <a href=https://gym.openai.com>OpenAI Gym</a> это набор инструментов для разработки и сравнения алгоритмов обучения с подкреплением.

OpenAI Gym предоставляет простой и универсальный API ко многим средам с разными свойствами, как простым так и сложным:
* Классические задачи управления и игрушечные примеры, которые можно найти в учебниках и на которых демонстрируется работа алгоритмов обучения с подкреплением (в основном они будут использоваться в этом курсе)
* Игры Atari (оказали огромное влияние на достижения в обучении с подкреплением в последние годы)
* 2D и 3D среды для контроля роботов в симуляции (используют проприетарный движок <a href=http://www.mujoco.org/>MuJoCo</a>)

Запустим агента со случайными действиями в среде <a href=https://gym.openai.com/envs/CartPole-v0>CartPole-v0</a>. В этой среде агент должен удержать шест, который закреплен на тележке, в вертикальном положении. Агент может прикладывать силу *+1* или *-1* к тележке в попытке удержать шест. Эпизод игры заканчивается, если шест отклоняется от вертикального положения больше, чем на *15* градусов или тележка сдвигается больше, чем на *2.4* единицы расстояния от центра. За каждый шаг времени, когда шест находится в вертикальном положении, агент получает награду в размере *+1* очко. Задаче считается "решенной" при получении агентом средней за 100 попыток награды *195* очков.

#### Для работы нам потребуется установить gym и numpy. Gym следует установить командой pip install gym=0.9.2

In [1]:
# Импортируем необходимые библиотеки
import gym
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Image
%matplotlib inline

In [2]:
env = gym.make('CartPole-v0') # создаем среду CartPole-v0

In [3]:
env.reset()
for _ in range(1000):
    env.render()
    observation, reward, done, info = env.step(env.action_space.sample()) # агент выбирает случайные действия
    if done:
        env.reset()

In [4]:
env.close() # выключим визуализацию

Вспомним элементы проблемы обучения с подкреплением <img src="./scheme.png" width=500>

OpenAI Gym предоставляет такой же интерфейс взаимодействия со средой:
* Среда в ответ на действие агента предоставляет *observation (object)*- специфичный для конкретной среды объект, который предствляет наблюдения агента. Например, пиксели камеры, значения углов и скоростей сочленений робота или позиции агента и других объектов в среде.
* *reward (float)* - значение награды, полученной агентом в резултате совершенного действия
* *done (boolean)* - флаг обозначающий окончание эпизода. Например, эпизод заканчивается, когда шест слишком сильно отклонился или агент попал в прорубь в среде FrozenLake
* *info (dict)* - словарь, содержащий диагностическую информацию, которую можно использовать для отладки, но не для обучения агента. Обычно мы присваеваем значение *info* переменной по-умолчанию *_*

Теперь, когда мы познакомились с API OpenAI Gym, посмотрим, сколько очков награды сможет в среднем получить за 100 эпизодов агент, выбирающий случайные действия.

In [ ]:
total_reward = []
env = gym.make('CartPole-v0')
for episode in range(100):
    episode_reward = 0
    observation = env.reset()
    for t in range(100):
        env.render()
        action = env.action_space.sample()
        observation, reward, done, info = env.step(action)
        episode_reward += reward
        if done:
            print("Episode {} finished after {} timesteps".format(episode+1, t+1))
            break
    total_reward.append(episode_reward)

In [6]:
env.close()

In [7]:
print(np.mean(total_reward))

22.42


Наш "cлучайный" агент получает в среднем 20 очков за 100 эпизодов. Не очень впечатляет.

В предыдущием эксперименте агент выбирал случайное действие. Важными объектами в OpenAI Gym являются пространства состояний и действий.

In [8]:
env = gym.make('CartPole-v0')

In [9]:
print(env.action_space.__doc__)

A discrete space in :math:`\{ 0, 1, \dots, n-1 \}`. 
    
    Example::
    
        >>> Discrete(2)
        
    


In [10]:
print(env.action_space)

Discrete(2)


In [11]:
print(env.action_space.n)

2


In [12]:
print(env.observation_space.__doc__)

A box in R^n, i.e.each coordinate is bounded.
    
    There are two common use cases:
    
    * Identical bound for each dimension::
        >>> Box(low=-1.0, high=2.0, shape=(3, 4), dtype=np.float32)
        Box(3, 4)
        
    * Independent bound for each dimension::
        >>> Box(low=np.array([-1.0, -2.0]), high=np.array([2.0, 4.0]), dtype=np.float32)
        Box(2,)

    


In [13]:
print(env.observation_space)

Box(4,)


In [14]:
print(env.observation_space.shape)

(4,)


In [15]:
env.observation_space.high

array([4.8000002e+00, 3.4028235e+38, 4.1887903e-01, 3.4028235e+38],
      dtype=float32)

In [16]:
env.observation_space.low

array([-4.8000002e+00, -3.4028235e+38, -4.1887903e-01, -3.4028235e+38],
      dtype=float32)

## Value Iteration

На лекции мы рассмотрели, как мы можем выучить оптимальную политику, используя алгоритм Value Iteration, если нам известна динамика среды, а также если пространства состояний и действий не большие и дискретные.

Попробуем выучить оптимальную политику в среде <a href=https://gym.openai.com/envs/FrozenLake-v0>FrozenLake-v0</a>. Это простая среда с маленькими пространствами состояний и действий, а также с известной динамикой

Созданим среду и выведем её описание

In [17]:
env = gym.make('FrozenLake-v0')

In [18]:
print(env.env.__doc__)


    Winter is here. You and your friends were tossing around a frisbee at the park
    when you made a wild throw that left the frisbee out in the middle of the lake.
    The water is mostly frozen, but there are a few holes where the ice has melted.
    If you step into one of those holes, you'll fall into the freezing water.
    At this time, there's an international frisbee shortage, so it's absolutely imperative that
    you navigate across the lake and retrieve the disc.
    However, the ice is slippery, so you won't always move in the direction you intend.
    The surface is described using a grid like the following

        SFFF
        FHFH
        FFFH
        HFFG

    S : starting point, safe
    F : frozen surface, safe
    H : hole, fall to your doom
    G : goal, where the frisbee is located

    The episode ends when you reach the goal or fall in a hole.
    You receive a reward of 1 if you reach the goal, and zero otherwise.

    


Как видно среда представляет собой поле 4 на 4, по которому нужно добраться от начала (клетка *S*) до цели (клетка *G*). При этом среда является недетерменированный - с определенной вероятностью при совершения действия агент подскользнется и попадет не в ту клетку, в которую направлялся. Клетка *H* обозначает прорубь. Игра закначивается, когда агент попадает в клетку *G* или в клету *H*. Если агент проваливается в прорубь, то он получает награду *0*, если достигает клетки цели - *1*. 

Посмотрим, сколько в среднем очков награды за 100 эпизодов получит наш агент, если будет выполнять случайные действия.

In [19]:
env.seed(0); # from gym.spaces import prng; prng.seed(10) # установим сид для воспроизводимости результатов эксперимента

In [20]:
total_reward = []
for episode in range(100):
    episode_reward = 0
    observation = env.reset()
    for t in range(100):
        env.render()
        action = env.action_space.sample()
        observation, reward, done, _ = env.step(action)
        episode_reward += reward
        if done:
            print("Episode {} finished after {} timesteps".format(episode+1, t+1))
            break
    total_reward.append(episode_reward)

In [21]:
print(np.mean(total_reward))

0.03


Как видим, только в 3 эпизодах из 100 агену удалось добраться до цели.

In [22]:
env.reset()
for _ in range(100):
    env.render()
    action = env.action_space.sample() # take a random action
    observation, reward, done, _ = env.step(action)
    if done:
        break


SFFF
FHFH
FFFH
HFFG
  (Up)
SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG


Из среды OpenAI Gym мы можем получить элементы MDP (Markov Decision Process)

В env.env.P хранится двухуровневый словарь, в котором первый ключ является состояние, а второй - действием.
Клетки ассоциированыс индексами [0, 1, 2, ..., 15] слева направо и сверху вниз.

In [23]:
print(np.arange(16).reshape(4,4))

[[ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]
 [12 13 14 15]]


Индексы действией [0, 1, 2, 3] соответствуют движению на Запад, Юг, Восток и Север.
env.env.P[state][action] возвращает лист кортежей (probability, nextstate, reward). Например, состояние 0 - это начальное состояние и информация о веротностях перехода для s=0 и a=0 содержит:

In [24]:
env.env.P[0][0]

[(0.3333333333333333, 0, 0.0, False),
 (0.3333333333333333, 0, 0.0, False),
 (0.3333333333333333, 4, 0.0, False)]

Другой пример - состояние 5 сооветсвует проруби, и все действия в данном состоянии приводят к тому же состоянию с вероятностью 1 и наградой 0

In [25]:
for i in range(4):
    print("P[5][%i] =" % i, env.env.P[5][i])

P[5][0] = [(1.0, 5, 0, True)]
P[5][1] = [(1.0, 5, 0, True)]
P[5][2] = [(1.0, 5, 0, True)]
P[5][3] = [(1.0, 5, 0, True)]


Вспомним, что из себя представляет алгоритм Value Iteration <img src="./value_iteration.png" width="500">

Задание считается решенным, если агент доходит до цели в среднем в 70% эпизодов.

In [26]:
n_states = env.env.nS
n_actions = env.env.nA
print("Number of states: {}".format(n_states))
print("Number of actions: {}".format(n_actions))

Number of states: 16
Number of actions: 4


Напишем несклько вспомогательных функций.

Поскольку алгоритм Value Iteration возвращает нам оптимальную V-функцию, то нам необходимо извлекать из нее оптимальную политику (как указано в последней строке псевдокода алгоритма)

In [27]:
def vi_extract_policy(env, v, gamma=1.0, extra_target_state_reward=0.):
    n_states = env.nS
    n_actions = env.nA
    policy = np.zeros(n_states)

    for state in range(n_states):
        q = np.zeros(n_actions)
        for action in range(n_actions):
            psrd = np.array(env.P[state][action]) # [[probability], [state], [reward], [done]]
            q[action] = np.sum(psrd[:,0]*(
                psrd[:,2] + gamma*v[psrd[:,1].astype(np.int)] + psrd[:,3]*extra_target_state_reward))
        policy[state] = np.argmax(q)

    return policy

Также напишем функцию для оценки нашей найденной политики.

In [28]:
def vi_evaluate_policy(env, policy, gamma=1.0, n=100):
    total_reward = []
    for episode in range(n):
        episode_reward = 0
        observation = env.reset()
        step = 0
        for _ in range(100):
            env.render()
            action = int(policy[observation])
            observation, reward, done, _ = env.step(action)
            episode_reward += (gamma**step)*reward
            step += 1
            if done:
                break
        total_reward.append(episode_reward)
    return np.mean(total_reward)

Нам остается написать основную функцию, которая вернет оптимальную V-функцию.

In [29]:
def value_iteration(env, max_iterations=10000, gamma=1.0, extra_target_state_reward=0., epsilon=1e-4):
    n_states = env.nS
    n_actions = env.nA    
    v = np.zeros(n_states)

    for i in range(max_iterations):
        delta = np.zeros(n_states)
        for state in range(n_states):
            u = v[state]
            q = np.zeros(n_actions)
            for action in range(n_actions):
                psrd = np.array(env.P[state][action]) # [[probability], [state], [reward], [done]]
                q[action] = np.sum(psrd[:,0]*(
                    psrd[:,2] + gamma*v[psrd[:,1].astype(np.int)] + psrd[:,3]*extra_target_state_reward))
            v[state] = np.max(q)
            delta[state] = np.max((delta[state], np.abs(u - v[state])))
#             print(f'[value_iteration] v[{state}]={v[state]:.3f} delta={delta}')
        if np.all(delta < epsilon):
            break

    policy = vi_extract_policy(env, v, gamma, extra_target_state_reward)
    return (v, policy)

Теперь мы можем найти оптимальную V-функцию, извлечь из нее оптимальную политику и оцениь ее.

In [30]:
optimal_v, optimal_policy = value_iteration(env)

In [31]:
print(optimal_v.reshape(4,4))

[[0.82182145 0.82126109 0.82087163 0.82067347]
 [0.82199325 0.         0.52824715 0.        ]
 [0.82226231 0.82260733 0.76389785 0.        ]
 [0.         0.88171208 0.94085038 0.        ]]


In [32]:
print(optimal_policy.reshape(4,4))

[[0. 3. 3. 3.]
 [0. 0. 0. 0.]
 [3. 1. 0. 0.]
 [0. 2. 1. 0.]]


In [33]:
optimal_policy_score = vi_evaluate_policy(env, optimal_policy, n=100)

In [34]:
optimal_policy_score

0.78

По сравнению со "случайным" агентом, который доходил до цели в 3 случаях из 100, наша новая политика позволяет добирться до цели в ~70% эпизодов.

## Policy Iteration

Вспомним, что из себя представляет алгоритм Policy Iteration <img src="policy_iteration.png" width="500">

Напишем необходимые вспомогательные функции.

Начнем с основного цикла алгоритма, который вернет нам оптимальную политику.

In [35]:
def policy_iteration(env, max_iterations=100, gamma=1.0, extra_target_state_reward=0., epsilon=10e-4):
    n_states = env.nS
    n_actions = env.nA
    v = np.zeros(n_states)
    policy = np.random.choice(n_actions, size=(n_states))  # initialize a random policy

    for i in range(max_iterations):
        old_policy_v = pi_compute_policy_v(env, policy, max_iterations, gamma, extra_target_state_reward, epsilon)
        new_policy = pi_extract_policy(env, old_policy_v, gamma, extra_target_state_reward)
        if (np.all(policy == new_policy)):
            print (f'Policy-Iteration converged at step {i+1}')
            break
        policy = new_policy
    
    return (old_policy_v, policy)

А также еще раз напишем функцию для оценки найденной политики.

In [36]:
def pi_evaluate_policy(env, policy, gamma=1.0, n=100):
    total_reward = []
    for episode in range(n):
        episode_reward = 0
        observation = env.reset()
        step = 0
        for _ in range(100):
            env.render() 
            action = int(policy[observation])
            observation, reward, done, _ = env.step(action)
            episode_reward += (gamma**step)*reward
            step += 1
            if done:
#                 print(f'[pi_evaluate_policy] done@{step}')
                break
        total_reward.append(episode_reward)
    return np.mean(total_reward)

Остается написать 2 функции, которые используются в основном цикле алгоритма Policy Iteration согласно псевдокоду.

In [37]:
def pi_compute_policy_v(env, policy, max_iterations, gamma=1.0, extra_target_state_reward=0., epsilon=1e-4):
    n_states = env.nS
    v = np.zeros(n_states)
    
    for _ in range(max_iterations):
        delta = np.zeros(n_states)
        for state in range(n_states):
            u = v[state]
            psrd = np.array(env.P[state][policy[state]]) # [[probability], [state], [reward], [done]]
            v[state] = np.sum(psrd[:,0]*(
                psrd[:,2] + gamma*v[psrd[:,1].astype(np.int)] + psrd[:,3]*extra_target_state_reward))
            delta[state] = np.max((delta[state], np.abs(u - v[state])))
#             print(f'[pi_compute_policy_v] v[{state}]={v[state]:.3f}\nv=\n{v.reshape(4, 12)}\ndelta=\n{delta.reshape(4, 12)}')
        if np.all(delta < epsilon):
            break

    return v

In [38]:
def pi_extract_policy(env, v, gamma=1.0, extra_target_state_reward=0.):
    n_states = env.nS
    n_actions = env.nA
    policy = np.zeros(n_states)

    for state in range(n_states):
        q = np.zeros(n_actions)
        for action in range(n_actions):
            psrd = np.array(env.P[state][action]) # [[probability], [state], [reward], [done]]
            q[action] = np.sum(psrd[:,0]*(
                psrd[:,2] + gamma*v[psrd[:,1].astype(np.int)] + psrd[:,3]*extra_target_state_reward))
        policy[state] = np.argmax(q)
    return policy

Теперь мы также можем найти оптимальную V-функцию, извлечь из нее оптимальную политику и оцениь ее.

In [39]:
optimal_v, optimal_policy = policy_iteration(env)

Policy-Iteration converged at step 4


In [40]:
print(optimal_policy.reshape(4,4))

[[0. 3. 3. 3.]
 [0. 0. 0. 0.]
 [3. 1. 0. 0.]
 [0. 2. 1. 0.]]


In [ ]:
optimal_policy_score = pi_evaluate_policy(env, optimal_policy)

In [42]:
print(optimal_policy_score)

0.68


## Дополнительное задание

Сравнить поведение случайного агента с обученным вышеприведенными методами агентом в средах из OpenAI Gym с известной динамикой <a href="https://gym.openai.com/envs/Taxi-v2/">Taxi-v2</a> и CliffWalking. Посмотрите как ведет себя агент при использовании разных значений фактора дисконтирования. 

Ко всем средам OpenAi Gym можно получить доступ не только через *gym.make* но и через обычный импорт модулей. Среда CliffWalking ипортируется именно так. 

### _CliffWalking_

In [43]:
from gym.envs import toy_text

In [44]:
env = toy_text.CliffWalkingEnv()

In [45]:
print(env.__doc__)


    This is a simple implementation of the Gridworld Cliff
    reinforcement learning task.

    Adapted from Example 6.6 (page 106) from Reinforcement Learning: An Introduction
    by Sutton and Barto:
    http://incompleteideas.net/book/bookdraft2018jan1.pdf

    With inspiration from:
    https://github.com/dennybritz/reinforcement-learning/blob/master/lib/envs/cliff_walking.py

    The board is a 4x12 matrix, with (using Numpy matrix indexing):
        [3, 0] as the start at bottom-left
        [3, 11] as the goal at bottom-right
        [3, 1..10] as the cliff at bottom-center

    Each time step incurs -1 reward, and stepping into the cliff incurs -100 reward
    and a reset to the start. An episode terminates when the agent reaches the goal.
    


Особенность данной среды состоит в том, что в ней отсутствует явная награда за переход в терминальное состояние `[3, 11]`. А поскольку факт того, что состояние является терминальным, никак не учитывается в оригинальной версии обоих алгоритмов, это приводит к тому, что они не пытаются "вести" игрока в состояние `[3, 11]`.
Для решения данной проблемы я решил ввести дополнительный параметр `extra_target_state_reward` - дополнительная награда за переход в терминальное состояние, и учитывать 4-й элемент списка `env.P[state][action]` - флаг `done`.

#### Value iteration

In [46]:
gamma = 1.

In [47]:
optimal_v, optimal_policy = value_iteration(env, gamma=gamma, extra_target_state_reward=1.)

In [48]:
print(optimal_policy.reshape(4,12))

[[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 2.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 2.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 2.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1.]]


Динамика среды детерминированная, поэтому достаточно одного прогона.

In [49]:
optimal_policy_score = vi_evaluate_policy(env, optimal_policy, gamma=gamma, n=1)

o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
x  C  C  C  C  C  C  C  C  C  C  T

o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
x  o  o  o  o  o  o  o  o  o  o  o
o  C  C  C  C  C  C  C  C  C  C  T

o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
o  x  o  o  o  o  o  o  o  o  o  o
o  C  C  C  C  C  C  C  C  C  C  T

o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
o  o  x  o  o  o  o  o  o  o  o  o
o  C  C  C  C  C  C  C  C  C  C  T

o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  x  o  o  o  o  o  o  o  o
o  C  C  C  C  C  C  C  C  C  C  T

o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  x  o  o  o  o  o  o  o
o  C  C  C  C  C  C  C  C  C  C  T

o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  x  o  o  o  o  o  o
o  C  C  C  C  C  C  C  C  C  C  T

o  o  o  o  o

In [50]:
print(optimal_policy_score)

-13.0


#### Policy iteration

По мере уменьшения $\gamma$ сходится относительно быстрее.

In [51]:
gamma = 1.

In [52]:
%%time

optimal_v, optimal_policy = policy_iteration(env, gamma=gamma, extra_target_state_reward=10.)

Policy-Iteration converged at step 15
Wall time: 2.63 s


In [53]:
print(optimal_policy.reshape(4, 12))

[[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 2.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 2.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 2.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1.]]


In [54]:
gamma = 0.95

In [55]:
%%time

optimal_v, optimal_policy = policy_iteration(env, gamma=gamma, extra_target_state_reward=10.)

Policy-Iteration converged at step 14
Wall time: 1.85 s


In [56]:
print(optimal_policy.reshape(4, 12))

[[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 2.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 2.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 2.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1.]]


In [57]:
optimal_policy_score = pi_evaluate_policy(env, optimal_policy, gamma=gamma, n=1)

o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
x  C  C  C  C  C  C  C  C  C  C  T

o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
x  o  o  o  o  o  o  o  o  o  o  o
o  C  C  C  C  C  C  C  C  C  C  T

o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
o  x  o  o  o  o  o  o  o  o  o  o
o  C  C  C  C  C  C  C  C  C  C  T

o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
o  o  x  o  o  o  o  o  o  o  o  o
o  C  C  C  C  C  C  C  C  C  C  T

o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  x  o  o  o  o  o  o  o  o
o  C  C  C  C  C  C  C  C  C  C  T

o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  x  o  o  o  o  o  o  o
o  C  C  C  C  C  C  C  C  C  C  T

o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  x  o  o  o  o  o  o
o  C  C  C  C  C  C  C  C  C  C  T

o  o  o  o  o

In [58]:
print(optimal_policy_score)

-9.733158334409895


### _Taxi-v2_

In [59]:
env = toy_text.TaxiEnv()

In [60]:
print(env.__doc__)


    The Taxi Problem
    from "Hierarchical Reinforcement Learning with the MAXQ Value Function Decomposition"
    by Tom Dietterich

    Description:
    There are four designated locations in the grid world indicated by R(ed), B(lue), G(reen), and Y(ellow). When the episode starts, the taxi starts off at a random square and the passenger is at a random location. The taxi drive to the passenger's location, pick up the passenger, drive to the passenger's destination (another one of the four specified locations), and then drop off the passenger. Once the passenger is dropped off, the episode ends.

    Observations: 
    There are 500 discrete states since there are 25 taxi positions, 5 possible locations of the passenger (including the case when the passenger is the taxi), and 4 destination locations. 
    
    Actions: 
    There are 6 discrete deterministic actions:
    - 0: move south
    - 1: move north
    - 2: move east 
    - 3: move west 
    - 4: pickup passenger
    - 5: dro

Здесь особенность среды в том, что при $\gamma = 1$ алгоритмы сходятся плохо, поэтому желательно выбирать $\gamma < 1$.

#### Value iteration

In [61]:
gamma = 0.95

In [62]:
optimal_v, optimal_policy_vi = value_iteration(env, gamma=gamma)

In [63]:
print(optimal_policy_vi)

[4. 4. 4. 4. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 5. 0. 0. 0. 3. 3. 3. 3.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 3. 0. 0. 0. 0. 0. 0. 0. 2. 2. 2. 2.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 2. 0. 0. 0. 0. 0. 0. 2. 2. 2. 2. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 2. 0. 0. 0. 0. 0. 0. 4. 4. 4. 4. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 5. 0. 0. 1. 1. 1. 1. 2. 2. 2. 2. 0. 0. 0. 0. 0. 0. 0. 0. 1. 2. 0. 0.
 1. 1. 1. 1. 2. 2. 2. 2. 0. 0. 0. 0. 0. 0. 0. 0. 1. 2. 0. 0. 3. 3. 3. 3.
 2. 2. 2. 2. 0. 0. 0. 0. 0. 0. 0. 0. 3. 2. 0. 0. 3. 3. 3. 3. 2. 2. 2. 2.
 0. 0. 0. 0. 0. 0. 0. 0. 3. 2. 0. 0. 3. 3. 3. 3. 1. 1. 1. 1. 0. 0. 0. 0.
 0. 0. 0. 0. 3. 1. 0. 0. 1. 1. 1. 1. 2. 2. 2. 2. 0. 0. 0. 0. 2. 2. 2. 2.
 1. 2. 0. 2. 1. 1. 1. 1. 2. 2. 2. 2. 3. 3. 3. 3. 2. 2. 2. 2. 1. 2. 3. 2.
 1. 1. 1. 1. 2. 2. 2. 2. 3. 3. 3. 3. 2. 2. 2. 2. 1. 2. 3. 2. 1. 1. 1. 1.
 2. 2. 2. 2. 3. 3. 3. 3. 0. 0. 0. 0. 1. 2. 3. 0. 1. 1. 1. 1. 1. 1. 1. 1.
 3. 3. 3. 3. 0. 0. 0. 0. 1. 1. 3. 0. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0.
 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 2. 2. 2. 2. 1.

In [ ]:
optimal_policy_score = vi_evaluate_policy(env, optimal_policy_vi, gamma=gamma, n=100)

In [65]:
print(optimal_policy_score)

1.892240737240751


#### Policy iteration

In [66]:
gamma = 0.95

In [67]:
optimal_v, optimal_policy_pi = policy_iteration(env, gamma=gamma)

Policy-Iteration converged at step 15


In [68]:
optimal_policy_pi

array([4., 4., 4., 4., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 5.,
       0., 0., 0., 3., 3., 3., 3., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 3., 0., 0., 0., 0., 0., 0., 0., 2., 2., 2., 2., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 2., 0., 0., 0., 0., 0., 0., 2., 2., 2., 2.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 2., 0., 0., 0., 0., 0., 0., 4.,
       4., 4., 4., 0., 0., 0., 0., 0., 0., 0., 0., 0., 5., 0., 0., 1., 1.,
       1., 1., 2., 2., 2., 2., 0., 0., 0., 0., 0., 0., 0., 0., 1., 2., 0.,
       0., 1., 1., 1., 1., 2., 2., 2., 2., 0., 0., 0., 0., 0., 0., 0., 0.,
       1., 2., 0., 0., 3., 3., 3., 3., 2., 2., 2., 2., 0., 0., 0., 0., 0.,
       0., 0., 0., 3., 2., 0., 0., 3., 3., 3., 3., 2., 2., 2., 2., 0., 0.,
       0., 0., 0., 0., 0., 0., 3., 2., 0., 0., 3., 3., 3., 3., 1., 1., 1.,
       1., 0., 0., 0., 0., 0., 0., 0., 0., 3., 1., 0., 0., 1., 1., 1., 1.,
       2., 2., 2., 2., 0., 0., 0., 0., 2., 2., 2., 2., 1., 2., 0., 2., 1.,
       1., 1., 1., 2., 2.

In [69]:
assert(all(optimal_policy_vi == optimal_policy_pi))

In [ ]:
optimal_policy_score = pi_evaluate_policy(env, optimal_policy_pi, gamma=gamma, n=100)

In [71]:
print(optimal_policy_score)

2.0451493991495777
